# Notebook Setup
This notebook is supposed to be launched in Google Colaboratory. Just point the repo from colab and then go!

In [1]:
import tensorflow as tf
import tensorflow.keras as K
import os
import cv2
import numpy as np

['/build/opencv/src/build/lib/python3', '', '/home/giovi/backup/machine_learning/projects/enet_tensorflow/notebooks', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/lib/python3.7/site-packages', '/usr/lib/python3.7/site-packages/IPython/extensions', '/home/giovi/.ipython']


ImportError: ERROR: recursion is detected during loading of "cv2" binary extensions. Check OpenCV installation.

In [ ]:
tf.__version__

In [ ]:
# Remove the comment below and clone the repo
# ! git clone https://github.com/gevero/enet_tensorflow.git

In [ ]:
# import encoders
import sys
# sys.path.append('../subclassing_api')  # Local path
sys.path.append('enet_tensorflow/subclassing_api')  # Colab path
import encoders_sub as enet

# Load Fashion MNIST Dataset

In [ ]:
# import dataset
mnist = K.datasets.fashion_mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# reshape dataset
x_train = x_train.reshape([-1,28,28,1])
x_test = x_test.reshape([-1,28,28,1])
x_train = np.array([cv2.resize(img,(32,32)) for img in x_train[:,:,:,0]]).reshape([-1,32,32,1])
x_test = np.array([cv2.resize(img,(32,32)) for img in x_test[:,:,:,0]]).reshape([-1,32,32,1])
y_train = K.utils.to_categorical(y_train)
y_test = K.utils.to_categorical(y_test)

### Data generator for data augmentation

In [ ]:
datagen = K.preprocessing.image.ImageDataGenerator(
                             rotation_range=5,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             horizontal_flip=True)

In [ ]:
datagen.fit(x_train)

# Fashion Mnist Baseline Model

In [ ]:
# defining standard and tpu model
input_layer = K.layers.Input(shape=(32,32,1))
output_layer = enet.fashion_mnist_baseline(input_layer)
fashion_mnist_baseline_model = K.Model(inputs=input_layer, outputs=output_layer)
tpu_baseline_model = tf.contrib.tpu.keras_to_tpu_model(
    fashion_mnist_baseline_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

In [ ]:
# Compile standard and tpu model
fashion_mnist_baseline_model.compile(optimizer=tf.train.AdamOptimizer(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
tpu_baseline_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['categorical_accuracy']
)

In [ ]:
fashion_mnist_baseline_model.summary()

### Run without Data Augmentation

In [ ]:
tpu_baseline_model.fit(x=x_train,y=y_train, batch_size=256,epochs=50,
                                 validation_data=(x_test,y_test))

### Run with Data Augmentation

In [ ]:
# defining standard and tpu model
input_layer = K.layers.Input(shape=(32,32,1))
output_layer = enet.fashion_mnist_baseline(input_layer)
fashion_mnist_baseline_model = K.Model(inputs=input_layer, outputs=output_layer)
tpu_baseline_model = tf.contrib.tpu.keras_to_tpu_model(
    fashion_mnist_baseline_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

In [ ]:
# Compile standard and tpu model
fashion_mnist_baseline_model.compile(optimizer=tf.train.AdamOptimizer(),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
tpu_baseline_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['categorical_accuracy']
)

In [ ]:
tpu_baseline_model.fit_generator(datagen.flow(x_train, y_train, batch_size=1024),epochs=50,
                          validation_data=(x_test,y_test))

# Fashion Mnist Enet Model

In [ ]:
# input layer
input_layer_enet = tf.keras.layers.Input(shape=(28,28,1))

In [ ]:
# mnist model enet
fashion_mnist_model_enet = enet.fashion_mnist_enc(dropout=0.25)
output_layer = fashion_mnist_model_enet(input_layer_enet)

In [ ]:
fashion_mnist_model_enet.summary()

### Run without Data Augmentation

In [ ]:
# lr_scheduler
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs',batch_size=1024)]

In [ ]:
# The compile step specifies the training configuration.
fashion_mnist_model_enet.compile(optimizer=tf.train.AdamOptimizer(learning_rate=3e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

In [ ]:
fashion_mnist_model_enet.fit(x=x_train,y=y_train, batch_size=1024,epochs=50,
                             validation_data=(x_test,y_test),
                             callbacks=callbacks)

### Run with Data Augmentation

In [ ]:
# mnist model enet
output_layer_enet = encoders.enet_encoder_mnist(input_layer_enet,dropout=0.20)
mnist_model_enet = tf.keras.Model(inputs=input_layer_enet, outputs=output_layer_enet)

In [ ]:
mnist_model_enet.summary()

In [ ]:
# lr_scheduler
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs',batch_size=1024)]

In [ ]:
# The compile step specifies the training configuration.
mnist_model_enet.compile(optimizer=tf.train.AdamOptimizer(learning_rate=3e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

In [ ]:
mnist_model_enet.fit_generator(datagen.flow(x_train, y_train, batch_size=1024),epochs=200,
                               validation_data=(x_test,y_test),
                               callbacks=callbacks)